In [1]:
import numpy as np
import os
import gc
import pickle
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential, model_from_json

Using Theano backend.


In [2]:
# Reconstructing saved model
model = model_from_json(open('my_model_architecture.json').read())
model.load_weights('my_model_weights.h5')
model.compile(optimizer='adadelta', loss='categorical_crossentropy')

In [4]:
# Ask user to input some text
raw_input_text = raw_input("Please enter the contect of your email.")


Please enter the contect of your email.this is not a real email


In [5]:
def review_to_words( raw_text ):
    # Function to convert a raw text to a string of words
    # The input is a single string , and 
    # the output is a single string
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_text, "lxml").get_text() 
    #
    # 2. Remove non-letters        
    #letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = review_text.lower().split()                            
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    #stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    #meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( words ))   

##### Pre-processing the text to be inputted to model

In [6]:
input_text_list = []
input_text_list.append(review_to_words(raw_input_text))
#input_text_list=review_to_words(raw_input_text)
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("bag_of_words_features.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(input_text_list))
tfidf = tfidf.toarray()
#tfidf = tfidf.astype('int64')

In [7]:
print tfidf.shape

(1, 8703)


In [8]:
prediction_results_class = model.predict_classes(np.asarray(tfidf), verbose=0)
prediction_results_prob = model.predict(np.asarray(tfidf), verbose=0)

In [9]:
print prediction_results_class
print prediction_results_prob

[0]
[[  9.99961019e-01   3.89723791e-05]]
